# MCP (Model Context Protocol) Implementation

From AI-Maker-Space/MCP-Event repository

In [ ]:
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
from tavily import TavilyClient
import os
from dice_roller import DiceRoller

load_dotenv()

mcp = FastMCP("websearch-server")
client = TavilyClient(os.getenv("TAVILY_API_KEY"))

@mcp.tool()
def web_search(query: str) -> str:
    """Search the web for information about the given query"""
    search_results = client.get_search_context(query=query)
    return search_results

@mcp.tool()
def roll_dice(notation: str, num_rolls: int = 1) -> str:
    """Roll the dice with the given notation"""
    roller = DiceRoller(notation, num_rolls)
    return str(roller)  

if __name__ == "__main__":
    mcp.run(transport="stdio")

In [ ]:
import random
import re

class DiceRoller:
    def __init__(self, notation, num_rolls=1):
        self.notation = notation
        self.num_rolls = num_rolls
        self.dice_pattern = re.compile(r"(\d+)d(\d+)(k(\d+))?")

    def roll_dice(self):
        match = self.dice_pattern.match(self.notation)
        if not match:
            raise ValueError("Invalid dice notation")

        num_dice = int(match.group(1))
        dice_sides = int(match.group(2))
        keep = int(match.group(4)) if match.group(4) else num_dice

        rolls = [random.randint(1, dice_sides) for _ in range(num_dice)]
        rolls.sort(reverse=True)
        kept_rolls = rolls[:keep]

        return rolls, kept_rolls

    def roll_multiple(self):
        """Roll the dice multiple times according to num_rolls"""
        results = []
        for _ in range(self.num_rolls):
            rolls, kept_rolls = self.roll_dice()
            results.append({
                "rolls": rolls,
                "kept": kept_ro

In [ ]:
import numpy as np
import re

class DiceRoller:
    def __init__(self, notation, num_rolls=1):
        self.notation = notation
        self.num_rolls = num_rolls
        self.dice_pattern = re.compile(r"(\d+)d(\d+)(k(\d+))?")

    def roll_dice(self):
        match = self.dice_pattern.match(self.notation)
        if not match:
            raise ValueError("Invalid dice notation")

        num_dice = int(match.group(1))
        dice_sides = int(match.group(2))
        keep = int(match.group(4)) if match.group(4) else num_dice

        # Use numpy to generate random integers
        rolls = np.random.randint(1, dice_sides + 1, size=num_dice).tolist()
        rolls.sort(reverse=True)
        kept_rolls = rolls[:keep]

        return rolls, kept_rolls

    def roll_multiple(self):
        """Roll the dice multiple times according to num_rolls"""
        results = []
        for _ in range(self.num_rolls):
            rolls, kept_rolls = self.roll_dice()
            results.append({
      

# A2A (Agent-to-Agent) Protocol Implementation

From AI-Maker-Space/AIM-A2A-Event repository

In [ ]:
import hashlib
import json
import logging
import time
import uuid

from typing import Any

import httpx
import jwt

from jwcrypto import jwk
from jwt import PyJWK, PyJWKClient
from starlette.requests import Request
from starlette.responses import JSONResponse


logger = logging.getLogger(__name__)
AUTH_HEADER_PREFIX = 'Bearer '


class PushNotificationAuth:
    def _calculate_request_body_sha256(self, data: dict[str, Any]):
        """Calculates the SHA256 hash of a request body.

        This logic needs to be same for both the agent who signs the payload and the client verifier.
        """
        body_str = json.dumps(
            data,
            ensure_ascii=False,
            allow_nan=False,
            indent=None,
            separators=(',', ':'),
        )
        return hashlib.sha256(body_str.encode()).hexdigest()


class PushNotificationSenderAuth(PushNotificationAuth):
    def __init__(self):
        self.public_keys = []
        self.private_key_jwk: PyJWK = None

   

In [ ]:
import asyncio
import threading
import traceback

from push_notification_auth import PushNotificationReceiverAuth
from starlette.applications import Starlette
from starlette.requests import Request
from starlette.responses import Response


class PushNotificationListener:
    def __init__(
        self,
        host,
        port,
        notification_receiver_auth: PushNotificationReceiverAuth,
    ):
        self.host = host
        self.port = port
        self.notification_receiver_auth = notification_receiver_auth
        self.loop = asyncio.new_event_loop()
        self.thread = threading.Thread(
            target=lambda loop: loop.run_forever(), args=(self.loop,)
        )
        self.thread.daemon = True
        self.thread.start()

    def start(self):
        try:
            # Need to start server in separate thread as current thread
            # will be blocked when it is waiting on user prompt.
            asyncio.run_coroutine_threadsafe(
                self.start_ser

In [ ]:
import asyncio
import base64
import os
import urllib
import httpx

from uuid import uuid4

import asyncclick as click
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich.json import JSON
from rich.markdown import Markdown

from a2a.client import A2AClient, A2ACardResolver
from a2a.types import (
    Part,
    TextPart,
    FilePart,
    FileWithBytes,
    Task,
    TaskState,
    Message,
    TaskStatusUpdateEvent,
    TaskArtifactUpdateEvent,
    MessageSendConfiguration,
    SendMessageRequest,
    SendStreamingMessageRequest,
    MessageSendParams,
    GetTaskRequest,
    TaskQueryParams,
    JSONRPCErrorResponse,
)
from push_notification_auth import PushNotificationReceiverAuth

# Initialize Rich console
console = Console()

def extract_text_from_parts(parts):
    """Extract text content from message parts."""
    texts = []
    for part in parts:
        # Handle the nested Part structure: Part(root=TextPart(...))
        if hasattr(p